# 深入 Deep Learning

## Convolution Neural Networks

卷积神经网络 (CNNs)在计算机视觉任务中获得了巨大的成功。一个卷积神经网络神经网络由以下部分构成：

* Convolution layers
* Pooling layers
* Dense layers (for final prediction)

在本次实验中, 我们将运用 CNNs 来学习 Cifar10 数据. 像数据集 MNIST一样, Cifar10 是计算机视觉中的另一个数据集. 通过该数据集，我们将学习：

* 如何做简单的数据探索
* 如何定义一个 CNN 模型
* 如何形式化数据满足训练模型需要
* 如何训练一个模型



## 第一步导入相关的库

In [ ]:
import random
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
%matplotlib inline

def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
fix_random_seed(4321)
print("TensorFlow version: {}".format(tf.__version__))

## 第二步 加载和探索数据

首先使用`tensorflow-datasets` 库加载数据集，然后输出数据集中的每个第一项，查看数据内容，将看到图像数据在[0,255]之间变化。最后，我们将绘制一些图像，以理解我们必须使用的类和图像。

In [ ]:
# Section 4.2

import tensorflow_datasets as tfds
# Loading the CIFAR10 dataset
data = tfds.load('cifar10')

In [ ]:
print(data)

In [ ]:
# Section 4.2

# Let's see the first element in the training set
for i in data["train"].take(1):
    print(i)

## 第三步 数据展示

绘制一些数据以及这些图像类别

In [ ]:
# Section 4.2

import matplotlib.pyplot as plt
%matplotlib inline

# Take 10 samples randomly to plot
sample_images, sample_labels = [],[]
for d in data["train"].shuffle(100, seed=4321).take(10):
    sample_images.append(d["image"].numpy())
    sample_labels.append(d["label"].numpy())

# Creating a label map mapping the integer label to the string
label_map = dict(zip(
    list(range(10)),
    ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]))

# Plotting the images
f, axes = plt.subplots(2, 5, figsize=(9,4))
for i, (img, lbl) in enumerate(zip(sample_images, sample_labels)):
    r, c = i//5, i%5
    axes[r,c].imshow(img,cmap='gray')
    axes[r,c].axis('off')
    axes[r,c].set_title("Label: {}".format(label_map[lbl]))

## 第四步 定义模型

定义一个CNN模型：我们试图运行这个模型时会出现一个错误。这是因为，我们需要注意模型每一层的输出大小。在这种情况下，我们创建的模型导致了无效的高度和宽度维度 **causing an error**.

In [ ]:
# Section 4.2
# Code listing 4.2

from tensorflow.keras import layers, models
import tensorflow.keras.backend as K

K.clear_session()

# Defining the CNN model
cnn = models.Sequential(
    [layers.Conv2D(filters=16,kernel_size=(9,9), strides=(2,2), activation='relu', input_shape=(32,32,3)), # 32->12
     layers.Conv2D(32, (7,7), activation='relu'), # 12 -> 6
     layers.Conv2D(64, (7,7), activation='relu'), # 6 -> -1
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')]
)

## 第五步 以正确的方式定义模型

解决这个错误，修正上面的模型，这样我们就不会有任何错误。

In [ ]:
# Section 4.2
# Code listing 4.3

from tensorflow.keras import layers, models
import tensorflow.keras.backend as K

K.clear_session()

# Defining the CNN model without any errores
cnn = models.Sequential(
    [layers.Conv2D(filters=16,kernel_size=(3,3), strides=(2,2), activation='relu', padding='same', input_shape=(32,32,3)), # 32->16
     layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'), # 16->8
     layers.Conv2D(32, (3,3), activation='relu', padding='same'), # 8 -> 8
     layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'), # 8->4
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(32, activation='relu'),
     layers.Dense(10, activation='softmax')]
)

# Compiling the model
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

## 第六步 训练模型
最后，可以训练模型，需要稍微改变数据集对象，把图像转换为' float32 '(原来是' uint8 ')，并将标签转换为one-hot编码向量。为此，将使用' tf.data.DataSet.map() '函数。

然后训练模型，模型的训练精度在稳步增长。

In [ ]:
# Section 4.2

import tensorflow as tf

def format_data(x, depth):
    """ Create a tuple where 1st element is a batch of images 
    and the second is a batch of onehot encoded vectors"""
    return (tf.cast(x["image"], 'float32'), tf.one_hot(x["label"], depth=depth))

# Map the dataset using the function
tr_data = data["train"].map(lambda x: format_data(x, depth=10)).batch(32)

In [ ]:
for d in tr_data.take(1):
    print(d)

In [ ]:
# Section 4.2
# Fit the data
history = cnn.fit(tr_data,epochs=25)